### Notebook for concept detection in neural network

In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import numpy as np
import os
import sys
from tqdm import tqdm
import json
import random
import string

sys.path.append(os.path.abspath(os.path.join(os.path.pardir, 'src')))

from concepts import static_concepts, linear_regression
import env
from policy import ConvNet

model_name = "net"
session_name = "endeavor"
board_size = 5
board_name = f'{board_size}x{board_size}'

agents_to_sample = [0, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]

full_model_path = f"../models/saved_sessions/board_size_{board_size}/{session_name}/"

CONCEPT_FUNC = static_concepts.play_center_in_opening

CONCEPT_NAME = static_concepts.play_center_in_opening.__name__

CASES_TO_SAMPLE = 2500 # 25000


In [12]:
def load_model(full_name, model_name, epoch):
    model_path = full_name + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, model_path)
    return model

agents = [load_model(full_model_path, model_name, epoch) for epoch in agents_to_sample]

In [13]:
def play_match(agents: list[ConvNet], board_size, concept_function):
    go_env = env.GoEnv(board_size)
    go_env.reset()

    player_to_start = 1 if np.random.random() > 0.5 else 0
    current_player = player_to_start

    total_moves = board_size * board_size * 4
    moves = 0
    random_moves = False

    positive_cases = []
    negative_cases = []

    sample_ratio = 0.8
    alpha = 1

    game_over = False

    prev_turn_state = np.zeros((board_size, board_size))
    temp_prev_turn_state = np.zeros((board_size, board_size))
    prev_opposing_state = np.zeros((board_size, board_size))

    while not game_over:
        if moves == total_moves:
            break

        state = go_env.canonical_state()

        valid_moves = go_env.valid_moves()

        if current_player == 0:
            state_copy = np.array([state[0], prev_turn_state, state[1], prev_opposing_state, np.zeros((board_size, board_size))])
        else:
            state_copy = np.array([state[0], prev_turn_state, state[1], prev_opposing_state, np.ones((board_size, board_size))])
    
        if np.random.random() < sample_ratio:
            pos = concept_function(state)
            if pos:
                positive_cases.append(state_copy)
            elif not pos:
                negative_cases.append(state_copy)

        if random_moves:
            action = go_env.uniform_random_action()
        else:
            action = agents[current_player].best_action(state_copy, valid_moves, alpha=alpha)

        _, _, game_over, _ = go_env.step(action)

        moves += 1

        current_player = 1 - current_player
        # Update the previous state
        prev_turn_state = temp_prev_turn_state
        prev_opposing_state = state[0]
        temp_prev_turn_state = prev_opposing_state
    
    return positive_cases, negative_cases

positive_cases = []
negative_cases = []

positive_bar = tqdm(total=CASES_TO_SAMPLE, desc="Positive cases")

while len(positive_cases) < CASES_TO_SAMPLE or len(negative_cases) < CASES_TO_SAMPLE:
    for i in range(len(agents)):
        for j in range(i + 1, len(agents)):
            pos, neg = play_match([agents[i], agents[j]], board_size, CONCEPT_FUNC)

            positive_cases.extend(pos)
            negative_cases.extend(neg)

            positive_bar.update(len(pos))

positive_cases = positive_cases[:CASES_TO_SAMPLE]
negative_cases = negative_cases[:CASES_TO_SAMPLE]

# Create numpy arrays
positive_cases = np.array(positive_cases)
negative_cases = np.array(negative_cases)

Positive cases:   0%|          | 0/2500 [02:18<?, ?it/s]


Positive cases: 2503it [01:18, 34.13it/s]                          

Positive cases: 2503it [01:34, 34.13it/s]

In [14]:
# Print shapes
print("Positive cases: ", positive_cases.shape)
print("Negative cases: ", negative_cases.shape)

Positive cases:  (2500, 5, 5, 5)
Negative cases:  (2500, 5, 5, 5)


In [15]:
# Test if there are any duplicates accross the two sets
for i in range(len(positive_cases)):
    for j in range(len(negative_cases)):
        if np.array_equal(positive_cases[i], negative_cases[j]):
            print("Duplicate found!")
            print(positive_cases[i])
            print(negative_cases[j])
            break

In [16]:
# Find the number of any duplicates within the positive cases
duplicate_count = 0
for i in range(len(positive_cases)):
    for j in range(i + 1, len(positive_cases)):
        if np.array_equal(positive_cases[i], positive_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in positive cases: ", duplicate_count)

# Find the number of any duplicates within the negative cases
duplicate_count = 0
for i in range(len(negative_cases)):
    for j in range(i + 1, len(negative_cases)):
        if np.array_equal(negative_cases[i], negative_cases[j]):
            duplicate_count += 1
            break

print("Duplicate count in negative: ", duplicate_count)

Duplicate count in positive cases:  2150
Duplicate count in negative:  292


In [ ]:
# Print all the positive cases
for i in range(len(positive_cases)):
    print(f"Positive case {i}:")
    print(positive_cases[i])
    print()

In [18]:
# Positions to consider are 80% of the total positions
POSITIONS_TO_CONSIDER = int(0.8 * positive_cases.shape[0] * 2)
print(f"Positions to consider: {POSITIONS_TO_CONSIDER}")
#POSITIONS_TO_CONSIDER = 4000 #40000
VALIDATION_POSITIONS = 10000 #10000

BINARY = True

Positions to consider: 4000


In [19]:
# First test if the concept can be regressed form the inputs
name = "input"
all_cases = np.concatenate([positive_cases, negative_cases])
all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
all_labels = np.array(all_labels)
shuffled_indices = np.arange(all_labels.shape[0])

np.random.shuffle(shuffled_indices)

all_cases = all_cases[shuffled_indices]
all_labels = all_labels[shuffled_indices]

points = all_cases.reshape(all_cases.shape[0], -1)

# Use the regression
score = linear_regression.perform_regression(
    points=points[:POSITIONS_TO_CONSIDER], 
    targets=all_labels[:POSITIONS_TO_CONSIDER], 
    validation_points=points[POSITIONS_TO_CONSIDER:], 
    validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
    is_binary=BINARY
)

print("Regression score: ", score)

# Remove the files if they exist
if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
    # Test if epoch folder exists
    if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
        # Remove all files in the epoch folder
        # Fist save the 
        for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name)):
            os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name, file))

os.makedirs("../concept_presences", exist_ok=True)
os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, name), exist_ok=True)

random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

# Save concept presences in json file
with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, name, random_suffix), "w") as f:
    json.dump(score, f)

Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.7432 - val_loss: 0.7199
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7083 - val_loss: 0.6872
Epoch 3/50
125/125 [==============================] - 0s 987us/step - loss: 0.6784 - val_loss: 0.6589
Epoch 4/50
125/125 [==============================] - 0s 988us/step - loss: 0.6519 - val_loss: 0.6335
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6283 - val_loss: 0.6111
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6075 - val_loss: 0.5914
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5891 - val_loss: 0.5740
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5727 - val_loss: 0.5584
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5580 - val_loss: 0.5444
Epoch 10/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5448 - val_loss: 0.

In [20]:
epochs_to_look_at = agents_to_sample

for epoch in epochs_to_look_at:
    path = full_model_path + model_name + "_" + str(epoch) + ".keras"
    model = ConvNet(board_size, path)

    # Will have a length equal to the sum of the numer of rows in the positive and negative cases arrays
    # And will contain 1s for positive cases and 0s for negative cases
    # Is used as labels/targets for the regression
    all_cases = np.concatenate([positive_cases, negative_cases])
    all_labels = [1] * positive_cases.shape[0] + [0] * negative_cases.shape[0]
    all_labels = np.array(all_labels)
    shuffled_indices = np.arange(all_labels.shape[0])

    np.random.shuffle(shuffled_indices)

    all_cases = all_cases[shuffled_indices]
    all_labels = all_labels[shuffled_indices]

    concept_presences = {}
    
    outputs = model.get_all_activation_values(all_cases)

    # Merge outputs
    merged_outputs = []
    for output_batch in outputs:
        for i, output_layer in enumerate(output_batch):
            if len(merged_outputs) <= i:
                merged_outputs.append([])
            merged_outputs[i].extend(output_layer)

    for i, layer_output in enumerate(merged_outputs):
        merged_outputs[i] = np.array(merged_outputs[i])
    
    outputs = merged_outputs

    # Perform regression
    concept_presence_per_layer = []
    for (i, output) in enumerate(outputs):
        points = output.reshape((output.shape[0], np.prod(output.shape[1:])))
        # So one has (n, k) samples where n is the number of positions, and k is the total number of activation values in layer i.
        print(f"Performing regression for layer {i}")
        score = linear_regression.perform_regression(
            points=points[:POSITIONS_TO_CONSIDER], 
            targets=all_labels[:POSITIONS_TO_CONSIDER], 
            validation_points=points[POSITIONS_TO_CONSIDER:], 
            validation_targets=all_labels[POSITIONS_TO_CONSIDER:], 
            is_binary=BINARY
        )
        concept_presence_per_layer.append(score)

        print(f"The presence of {CONCEPT_NAME} in resblock {i} is {score}")
    concept_presences[CONCEPT_NAME] = concept_presence_per_layer

    # Remove the files if they exist
    if os.path.exists("../concept_presences/{}/{}/{}/".format(board_name, session_name, CONCEPT_NAME)):
        # Test if epoch folder exists
        if os.path.exists("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
            # Remove all files in the epoch folder
            # Fist save the 
            for file in os.listdir("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch)):
                os.remove("../concept_presences/{}/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch, file))

    os.makedirs("../concept_presences", exist_ok=True)
    os.makedirs("../concept_presences/{}".format(board_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}".format(board_name, session_name), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME), exist_ok=True)
    os.makedirs("../concept_presences/{}/{}/{}/{}".format(board_name, session_name, CONCEPT_NAME, epoch), exist_ok=True)

    random_suffix = ''.join(random.choices(string.ascii_uppercase + string.digits, k=10))

    # Save concept presences in json file
    with open("../concept_presences/{}/{}/{}/{}/{}.json".format(board_name, session_name, CONCEPT_NAME, epoch, random_suffix), "w") as f:
        json.dump(concept_presences[CONCEPT_NAME], f)


Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 258.42it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 1.0213 - val_loss: 0.9551
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.9048 - val_loss: 0.8507
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8125 - val_loss: 0.7690
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7413 - val_loss: 0.7091
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6927 - val_loss: 0.6714
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6647 - val_loss: 0.6521
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6519 - val_loss: 0.6439
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6457 - val_loss: 0.6390
Epoch 9/50
125/125 [==============================] - 0s 964us/step - loss: 0.6412 - val_loss: 0.6348
Epoch 10/50
125/125 [==============================] - 0s 1ms/st

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 256.72it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.0323 - val_loss: 0.9401
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8817 - val_loss: 0.8110
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7697 - val_loss: 0.7146
Epoch 4/50
125/125 [==============================] - 0s 1ms/step - loss: 0.6839 - val_loss: 0.6394
Epoch 5/50
125/125 [==============================] - 0s 944us/step - loss: 0.6179 - val_loss: 0.5827
Epoch 6/50
125/125 [==============================] - 0s 931us/step - loss: 0.5708 - val_loss: 0.5442
Epoch 7/50
125/125 [==============================] - 0s 930us/step - loss: 0.5395 - val_loss: 0.5198
Epoch 8/50
125/125 [==============================] - 0s 960us/step - loss: 0.5198 - val_loss: 0.5049
Epoch 9/50
125/125 [==============================] - 0s 981us/step - loss: 0.5067 - val_loss: 0.4930
Epoch 10/50
125/125 [==============================] - 0

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 289.33it/s]


...

Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 1s 2ms/step - loss: 0.9673 - val_loss: 0.8398
Epoch 2/50
125/125 [==============================] - 0s 969us/step - loss: 0.7629 - val_loss: 0.6948
Epoch 3/50
125/125 [==============================] - 0s 965us/step - loss: 0.6515 - val_loss: 0.6073
Epoch 4/50
125/125 [==============================] - 0s 965us/step - loss: 0.5773 - val_loss: 0.5441
Epoch 5/50
125/125 [==============================] - 0s 979us/step - loss: 0.5217 - val_loss: 0.4951
Epoch 6/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4779 - val_loss: 0.4564
Epoch 7/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4432 - val_loss: 0.4257
Epoch 8/50
125/125 [==============================] - 0s 1ms/step - loss: 0.4154 - val_loss: 0.4010
Epoch 9/50
125/125 [==============================] - 0s 1ms/step - loss: 0.3940 - val_loss: 0.3828
Epoch 10/50
125/125 [==============================] - 0s 

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 293.86it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 0.9893 - val_loss: 0.8611
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7637 - val_loss: 0.7051
Epoch 3/50
125/125 [==============================] - 0s 968us/step - loss: 0.6424 - val_loss: 0.6107
Epoch 4/50
125/125 [==============================] - 0s 947us/step - loss: 0.5647 - val_loss: 0.5457
Epoch 5/50
125/125 [==============================] - 0s 973us/step - loss: 0.5083 - val_loss: 0.4959
Epoch 6/50
125/125 [==============================] - 0s 971us/step - loss: 0.4631 - val_loss: 0.4554
Epoch 7/50
125/125 [==============================] - 0s 944us/step - loss: 0.4275 - val_loss: 0.4237
Epoch 8/50
125/125 [==============================] - 0s 940us/step - loss: 0.3992 - val_loss: 0.3988
Epoch 9/50
125/125 [==============================] - 0s 968us/step - loss: 0.3777 - val_loss: 0.3800
Epoch 10/50
125/125 [==============================]

Getting activation outputs: 100%|██████████| 157/157 [00:00<00:00, 288.32it/s]


Performing regression for layer 0
Epoch 1/50
125/125 [==============================] - 0s 2ms/step - loss: 1.1617 - val_loss: 0.9947
Epoch 2/50
125/125 [==============================] - 0s 1ms/step - loss: 0.8781 - val_loss: 0.8073
Epoch 3/50
125/125 [==============================] - 0s 1ms/step - loss: 0.7300 - val_loss: 0.6960
Epoch 4/50
125/125 [==============================] - 0s 985us/step - loss: 0.6371 - val_loss: 0.6215
Epoch 5/50
125/125 [==============================] - 0s 1ms/step - loss: 0.5710 - val_loss: 0.5650
Epoch 6/50
125/125 [==============================] - 0s 969us/step - loss: 0.5194 - val_loss: 0.5189
Epoch 7/50
125/125 [==============================] - 0s 2ms/step - loss: 0.4774 - val_loss: 0.4823
Epoch 8/50
125/125 [==============================] - 0s 991us/step - loss: 0.4437 - val_loss: 0.4521
Epoch 9/50
125/125 [==============================] - 0s 983us/step - loss: 0.4160 - val_loss: 0.4265
Epoch 10/50
125/125 [==============================] - 0s 